<h3>Импорт библиотек</h3>

In [1]:
import pandas as pd
import matplotlib.pyplot  as plt
import os
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torch.utils.data as torch_data

<h3>Импорт датасета</h3>

In [10]:
df = pd.read_csv("data/Arctic_Boreal_CO2_Flux.csv",encoding='cp1252')
df.head()

,id,study_id,study_id_short,site_name,site_reference,country,latitude,longitude,start_date,end_date,...,notes_time_variant,citation,citation_data_overlap,data_contributor_or_author,email,orcid,data_availability,data_maturity,extraction_source,dataentry_person
0,1936,Adkinson_CA-WP2_tower1,Adkinson_CA-WP2_tower1,Alberta - Western Peatland - Poor Fen (Sphagnu...,CA-WP2,Canada,55.5375,-112.3343,2004-05-01,2004-05-31,...,originally positive GPP represented uptake but...,"Adkinson, A. C., Syed, K. H., & Flanagan, L. B...",NaN,Angela C. Adkinson,Lawrence B. Flanagan (larry.flanagan@uleth.ca),NaN,Published_Paper,Published,paper,Connon
1,1937,Adkinson_CA-WP2_tower1,Adkinson_CA-WP2_tower1,Alberta - Western Peatland - Poor Fen (Sphagnu...,CA-WP2,Canada,55.5375,-112.3343,2004-06-01,2004-06-30,...,originally positive GPP represented uptake but...,"Adkinson, A. C., Syed, K. H., & Flanagan, L. B...",NaN,Angela C. Adkinson,Lawrence B. Flanagan (larry.flanagan@uleth.ca),NaN,Published_Paper,Published,paper,Connon
2,1938,Adkinson_CA-WP2_tower1,Adkinson_CA-WP2_tower1,Alberta - Western Peatland - Poor Fen (Sphagnu...,CA-WP2,Canada,55.5375,-112.3343,2004-07-01,2004-07-31,...,originally positive GPP represented uptake but...,"Adkinson, A. C., Syed, K. H., & Flanagan, L. B...",NaN,Angela C. Adkinson,Lawrence B. Flanagan (larry.flanagan@uleth.ca),NaN,Published_Paper,Published,paper,Connon
3,1939,Adkinson_CA-WP2_tower1,Adkinson_CA-WP2_tower1,Alberta - Western Peatland - Poor Fen (Sphagnu...,CA-WP2,Canada,55.5375,-112.3343,2004-08-01,2004-08-31,...,originally positive GPP represented uptake but...,"Adkinson, A. C., Syed, K. H., & Flanagan, L. B...",NaN,Angela C. Adkinson,Lawrence B. Flanagan (larry.flanagan@uleth.ca),NaN,Published_Paper,Published,paper,Connon
4,1940,Adkinson_CA-WP2_tower1,Adkinson_CA-WP2_tower1,Alberta - Western Peatland - Poor Fen (Sphagnu...,CA-WP2,Canada,55.5375,-112.3343,2004-09-01,2004-09-30,...,originally positive GPP represented uptake but...,"Adkinson, A. C., Syed, K. H., & Flanagan, L. B...",NaN,Angela C. Adkinson,Lawrence B. Flanagan (larry.flanagan@uleth.ca),NaN,Published_Paper,Published,paper,Connon


In [11]:
df.columns

Index(['id', 'study_id', 'study_id_short', 'site_name', 'site_reference',
       'country', 'latitude', 'longitude', 'start_date', 'end_date',
       'meas_year', 'season', 'interval_month', 'start_day', 'end_day',
       'duration', 'biome', 'veg_type', 'veg_type_Short', 'veg_detail',
       'permafrost', 'disturbance', 'disturb_year', 'disturb_severity',
       'soil_moisture_class', 'site_activity', 'nee', 'gpp', 'reco',
       'ground_nee', 'ground_gpp', 'ground_reco', 'rsoil', 'flux_method',
       'flux_method_detail', 'measurement_frequency', 'diurnal_coverage',
       'spatial_reps_chamber', 'partition_method', 'gap_fill', 'gap_fill_perc',
       'tower_qa_qc_nee_flag', 'tower_qa_qc_source', 'method_error_nee',
       'method_error_technique', 'high_freq_availability',
       'aggregation_method', 'instrumentation', 'tower_version',
       'tower_data_restriction', 'tower_corrections',
       'spatial_variation_technique', 'light_response_method_chamber',
       'par_cutoff', '

Столбцов излишне много

In [50]:
df = df[['country', 'latitude', 'longitude', 'season', 'biome', 'disturbance', 'soil_moisture_class', 'nee', 'gpp', 'reco']]
df.head()

,country,latitude,longitude,season,biome,disturbance,soil_moisture_class,nee,gpp,reco
0,Canada,55.5375,-112.3343,spring,Boreal,NaN,Wet,-9.110169,-23.608017,14.699332
1,Canada,55.5375,-112.3343,summer,Boreal,NaN,Wet,-17.690678,-72.605790,53.006683
2,Canada,55.5375,-112.3343,summer,Boreal,NaN,Wet,-31.355932,-110.913140,78.841870
3,Canada,55.5375,-112.3343,summer,Boreal,NaN,Wet,-29.449152,-86.414250,56.570156
4,Canada,55.5375,-112.3343,autumn,Boreal,NaN,Wet,-20.550848,-41.425390,21.380846


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6309 entries, 0 to 6308
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              6309 non-null   object 
 1   latitude             6309 non-null   float64
 2   longitude            6309 non-null   float64
 3   season               6309 non-null   object 
 4   biome                6309 non-null   object 
 5   disturbance          1872 non-null   object 
 6   soil_moisture_class  3557 non-null   object 
 7   nee                  6309 non-null   float64
 8   gpp                  6309 non-null   float64
 9   reco                 6309 non-null   float64
dtypes: float64(5), object(5)
memory usage: 493.0+ KB


In [52]:
m_dict = {
    "country":{"Canada":0, "Finland":1, "Greenland":2, "Iceland":3, "Mongolia":4, "Norway":5, "Russia":6, "Sweeden":7, "USA":8},
    "season":{"winter":0, "spring":1, "summer":2, "autumn":3},
    "biome":{"Boreal":0, "Tundra":1},
    "disturbance":{"NA":0, "No":1, "Drainage":2, "Drought":3, "Fire":4, "Harvest":5, "Larval Outbreak":6, "Thermokarst":7},
    "soil_moisture_class":{"Dry":0, "Wet":1, "NA":2}
}

In [53]:
df_for_analyse = df.replace(m_dict)
df_for_analyse['disturbance'] = df_for_analyse['disturbance'].fillna(0)
df_for_analyse = df_for_analyse[df_for_analyse['soil_moisture_class'].notna()]
df_for_analyse.head()

,country,latitude,longitude,season,biome,disturbance,soil_moisture_class,nee,gpp,reco
0,0,55.5375,-112.3343,1,0,0.0,1.0,-9.110169,-23.608017,14.699332
1,0,55.5375,-112.3343,2,0,0.0,1.0,-17.690678,-72.605790,53.006683
2,0,55.5375,-112.3343,2,0,0.0,1.0,-31.355932,-110.913140,78.841870
3,0,55.5375,-112.3343,2,0,0.0,1.0,-29.449152,-86.414250,56.570156
4,0,55.5375,-112.3343,3,0,0.0,1.0,-20.550848,-41.425390,21.380846


In [54]:
len(df_for_analyse)

3557

Целевым значением будет столбец soil_moisture_class

In [55]:
class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

def data_split(ts, param="1", train_percentage=0.67):
    timeseries = ts[ts.columns[1]].values.astype('float32')
    train_size = int(len(timeseries) * train_percentage)
    train, test = timeseries[:train_size], timeseries[train_size:]
    return train, test, timeseries

def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

In [56]:
train, test, total = data_split(df_for_analyse)
lookback = 1
x_train, y_train = create_dataset(train, lookback=lookback)
x_test, y_test = create_dataset(test, lookback=lookback)

In [57]:
model = AirModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = torch_data.DataLoader(torch_data.TensorDataset(x_train, y_train), shuffle=True, batch_size=8)

EPOCHS = 20

In [58]:
def train_model(model, X_train, X_test, y_train, y_test, n_epochs):
    y_pred_train = None
    y_pred_test = None

    for epoch in range(n_epochs):
        model.train()
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # Validation
        if epoch % 10 != 0:
            continue
        model.eval()
        with torch.no_grad():
            y_pred = model(X_train)
            y_pred_train = y_pred
            train_rmse = np.sqrt(loss_fn(y_pred, y_train))
            y_pred = model(X_test)
            y_pred_test = y_pred
            test_rmse = np.sqrt(loss_fn(y_pred, y_test))
        print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))
    return y_pred_train, y_pred_test, model

In [61]:
y_pred_train, y_pred_test, model = train_model(model, x_train, x_test, y_train, y_test, EPOCHS)

Epoch 0: train RMSE 1.7367, test RMSE 1.7248
Epoch 10: train RMSE 1.6495, test RMSE 1.6392


In [62]:
torch.save(model, f"./lstm_forecast_{EPOCHS}_epochs.pth")

Данные для графика валидации отсутствуют, демонстрация подсчёта метрик и построения графика будет в лабораторной №13